# Comparação de gêneros no OBRAS entre PALAVRAS e UDPIPE

## Abrindo corpoPALAVRAS e corpoOBRAS

In [ ]:
! wget https://www.linguateca.pt/acesso/corpos/obras/corpoPALAVRAS.gz
! wget https://www.linguateca.pt/acesso/corpos/obras/corpoOBRAS.gz

In [1]:
! gunzip -f corpoPALAVRAS.gz
! gunzip -f corpoOBRAS.gz

In [2]:
import re
with open("corpoPALAVRAS", encoding="latin-1") as f:
    PALAVRAS = f.read()
with open("corpoOBRAS", encoding="latin-1") as f:
    OBRAS = f.read()

## Criando estrutura de dados

In [3]:
PALAVRAS_obras = [{
        'obra': re.findall('<obra id="([^"]*)"', x),
        'tituloobra': re.findall('<tituloobra id="([^"]*)"', x),
        'autor': re.findall('<autor id="([^"]*)"', x),
        'sentences': [[{
                    'forma': z.split("\t")[0],
                    'lema': z.split("\t")[1],
                    'pos': z.split("\t")[2],
                    'genero': z.split("\t")[5]
                } for z in y.split("\n") if len(z.split("\t")) > 5
                      ] for y in re.findall('<u>(.*?)</u>', x, flags=re.DOTALL) if '\t' in y]
        } for x in re.findall('<obra.*?</obra>', PALAVRAS, flags=re.DOTALL)]

print(PALAVRAS_obras[0]['autor'])
print(PALAVRAS_obras[0]['obra'])
print(PALAVRAS_obras[0]['tituloobra'])
PALAVRAS_obras[0]['sentences'][0]

['Aluísio Azevedo']
['A_Mortalha_de_Alzira Prosa:romance AA 1894 naturalismo_realismo_romantismo masc ']
['A Mortalha de Alzira']


[{'forma': 'A', 'lema': 'o', 'pos': 'DET_artd', 'genero': 'F'},
 {'forma': 'MORTALHA', 'lema': 'mORTALHA', 'pos': 'N', 'genero': 'F'},
 {'forma': 'DE', 'lema': 'de', 'pos': 'PRP', 'genero': '0'},
 {'forma': 'ALZIRA', 'lema': 'Alzira', 'pos': 'PROP', 'genero': 'F'}]

In [4]:
OBRAS_obras = [{
        'obra': re.findall('<obra id="([^"]*)"', x),
        'tituloobra': re.findall('<tituloobra id="([^"]*)"', x),
        'autor': re.findall('<autor id="([^"]*)"', x),
        'sentences': [[{
                    'forma': z.split("\t")[0],
                    'lema': z.split("\t")[8],
                    'pos': z.split("\t")[9],
                    'genero': z.split("\t")[12]
                } for z in y.split("\n") if len(z.split("\t")) > 5
                      ] for y in re.findall('<u>(.*?)</u>', x, flags=re.DOTALL) if '\t' in y]
        } for x in re.findall('<obra.*?</obra>', OBRAS, flags=re.DOTALL)]

print(OBRAS_obras[0]['autor'])
print(OBRAS_obras[0]['obra'])
print(OBRAS_obras[0]['tituloobra'])
OBRAS_obras[0]['sentences'][0]

['Aluísio Azevedo']
['A_Mortalha_de_Alzira Prosa:romance AA 1894 naturalismo_realismo_romantismo masc ']
['A Mortalha de Alzira']


[{'forma': 'A', 'lema': 'o', 'pos': 'DET_artd', 'genero': 'F'},
 {'forma': 'MORTALHA', 'lema': 'mORTALHA', 'pos': 'N', 'genero': 'F'},
 {'forma': 'DE', 'lema': 'de', 'pos': 'PRP', 'genero': '0'},
 {'forma': 'ALZIRA', 'lema': 'Alzira', 'pos': 'PROP', 'genero': 'F'}]

## Alinhando PALAVRAS e OBRAS

### Número de obras no PALAVRAS e no OBRAS

In [5]:
print(len(PALAVRAS_obras))
print(len(OBRAS_obras))

311
307


### Buscar obras que existem em um e não no outro

In [6]:
obras_proibidas = [x for x in [y['tituloobra'] for y in PALAVRAS_obras] if x not in [z['tituloobra'] for z in OBRAS_obras]]
obras_proibidas += [x for x in [y['tituloobra'] for y in OBRAS_obras] if x not in [z['tituloobra'] for z in PALAVRAS_obras]]
print([x for x in [y['tituloobra'] for y in PALAVRAS_obras] if x not in [z['tituloobra'] for z in OBRAS_obras]])
print([x for x in [y['tituloobra'] for y in OBRAS_obras] if x not in [z['tituloobra'] for z in PALAVRAS_obras]])
PALAVRAS_obras = [x for x in PALAVRAS_obras if x['tituloobra'] not in obras_proibidas]
OBRAS_obras = [x for x in OBRAS_obras if x['tituloobra'] not in obras_proibidas]

print(len(PALAVRAS_obras))
print(len(OBRAS_obras))

[['Ao Entardecer'], ['A retirada da Laguna, narrativa de campanha'], ['Inocência'], ['No declínio']]
[]
307
307


### Buscar obras fora de ordem entre PALAVRAS e OBRAS

In [7]:
fora_de_ordem = False
for i in range(len(PALAVRAS_obras)):
    if PALAVRAS_obras[i]['tituloobra'] != OBRAS_obras[i]['tituloobra']:
        fora_de_ordem = True
        print(f"{PALAVRAS_obras[i]['tituloobra']} / {OBRAS_obras[i]['tituloobra']}")
        
if not fora_de_ordem:
    print("Não há obras fora de ordem")

Não há obras fora de ordem


### Buscar obras sem id

In [8]:
obras_proibidas = []
for obra in PALAVRAS_obras:
    if not obra['obra']:
        print(obra['tituloobra'])
        obras_proibidas.append(obra['tituloobra'])        

PALAVRAS_obras = [x for x in PALAVRAS_obras if x['tituloobra'] not in obras_proibidas]
OBRAS_obras = [x for x in OBRAS_obras if x['tituloobra'] not in obras_proibidas]

print(len(PALAVRAS_obras))
print(len(OBRAS_obras))

['Eterno!']
['Orai por ele!']
['Pobre Finoca!']
['Suje-se gordo!']
['Vênus! Divina Vênus!']
['Vinte Anos! Vinte Anos!']
['Viver!']
300
300


### Buscar obras sem sentença

In [9]:
obras_proibidas = []
for obra in PALAVRAS_obras:
    if not obra['sentences']:
        print(obra['tituloobra'])
        obras_proibidas.append(obra['tituloobra'])        

PALAVRAS_obras = [x for x in PALAVRAS_obras if x['tituloobra'] not in obras_proibidas]
OBRAS_obras = [x for x in OBRAS_obras if x['tituloobra'] not in obras_proibidas]

print(len(PALAVRAS_obras))
print(len(OBRAS_obras))

['A causa secreta']
['Adão e Eva']
['A desejada das gentes']
['A mulher de preto']
['Anedota do cabriolet']
['Ao acaso']
['A parasita azul']
['As bodas de Luis Duarte']
['Aurora sem dia']
['Confissões de uma viúva moça']
['Conto de escola']
['D Paula']
['Entre santos']
['Ernesto de tal']
['Evolução ']
['Frei Simão']
['Henrique Chaves']
['Henrique Lombaerts']
['Idéias do canário']
['Joaquim Serra']
['José de Alencar']
['Lágrimas de Xerxes']
['Linha reta e linha curva']
['Luís Soares']
['Marcha fúnebre']
['Maria Cora']
['Missa do galo']
['Miss Dollar']
['O Califa de Platina']
['O caso da vara']
['O cônego ou Metafísica do estilo']
['O dicionário']
['O diplomático']
['O empréstimo']
['O enfermeiro']
['O futuro dos argentinos']
['O passado']
['O relógio de ouro']
['O segredo de Augusta']
['O Velho Senado']
['Pai contra mãe']
['Papéis velhos']
['Pílades e Orestes']
['Ponto de Vista']
['Trio em Lá menor']
['Uma carta']
['Uma noite']
['Um apólogo']
['Umas férias']
['Uma visita de Alcebíades']

### Buscar obras com número de sentenças diferente

In [10]:
sentencas_diferentes = False
for i in range(len(PALAVRAS_obras)):
    if len(PALAVRAS_obras[i]['sentences']) != len(OBRAS_obras[i]['sentences']):
        sentencas_diferentes = True
        print(f"{i} - {PALAVRAS_obras[i]['tituloobra']}")
        print(f"PALAVRAS ({len(PALAVRAS_obras[i]['sentences'])}) / ACDC ({len(OBRAS_obras[i]['sentences'])})")
        
if not sentencas_diferentes:
    print('Não há obras com número de sentenças diferente')

Não há obras com número de sentenças diferente


### Buscar sentenças com números de tokens diferentes e eliminar

In [11]:
output = ""
sentences_inequal = {}
diff_list = {'OBRAS': {}, 'PALAVRAS': {}}

for i in range(len(OBRAS_obras)):
    sentences_inequal[i] = []
    for k in range(len(OBRAS_obras[i]['sentences'])):
        text_PALAVRAS = " ".join([x['forma'] for x in PALAVRAS_obras[i]['sentences'][k]])
        text_OBRAS = " ".join([x['forma'] for x in OBRAS_obras[i]['sentences'][k]])
        if len(text_PALAVRAS.split()) != len(text_OBRAS.split()):
            output += f"{i}-{k}:"
            output += "\n" + text_PALAVRAS
            output += "\n" + text_OBRAS
            output += "\n\n"
            lista_PALAVRAS = [x['forma'] for x in PALAVRAS_obras[i]['sentences'][k]]
            lista_OBRAS = [x['forma'] for x in OBRAS_obras[i]['sentences'][k]]
            for diff in lista_PALAVRAS:
                if diff not in lista_OBRAS:
                    diff_list['OBRAS'][diff] = diff_list['OBRAS'][diff] + 1 if diff in diff_list['OBRAS'] else 1
            for diff in lista_OBRAS:
                if diff not in lista_PALAVRAS:
                    diff_list['PALAVRAS'][diff] = diff_list['PALAVRAS'][diff] + 1 if diff in diff_list['PALAVRAS'] else 1
            sentences_inequal[i].append(k)

numero_sentences_pre = sum([len(x['sentences']) for x in OBRAS_obras])
print(f"Total de sentenças no corpus: {numero_sentences_pre}")
for i in range(len(PALAVRAS_obras)):
    PALAVRAS_obras[i]['sentences'] = [x for k, x in enumerate(PALAVRAS_obras[i]['sentences']) if k not in sentences_inequal[i]]
    OBRAS_obras[i]['sentences'] = [x for k, x in enumerate(OBRAS_obras[i]['sentences']) if k not in sentences_inequal[i]]
numero_sentences_pos = sum([len(x['sentences']) for x in OBRAS_obras])
print(f"Sentenças com tamanho diferente, que foram suprimidas do experimento (salvas em lista_suprimidas.txt): {numero_sentences_pre - numero_sentences_pos}")
print(f"Total de sentenças no corpus sem as suprimidas: {numero_sentences_pos}")

import pprint
for corpus in diff_list:
    print(f"\n{len(diff_list[corpus])} palavras que foram suprimidas de {corpus}\n")
    pprint.pprint(sorted(diff_list[corpus].items(), key=lambda x: x[1], reverse=True))
print(f"\n{output}")
with open("lista_suprimidas.txt", "w") as f:
    f.write(output)

Total de sentenças no corpus: 393282
Sentenças com tamanho diferente, que foram suprimidas do experimento (salvas em lista_suprimidas.txt): 4
Total de sentenças no corpus sem as suprimidas: 393278

4 palavras que foram suprimidas de OBRAS

[('ia', 2), ('lo', 1), ('íamos', 1), ('Perorou', 1)]

0 palavras que foram suprimidas de PALAVRAS

[]

42-31:
Trá- lo íamos , eu e tu , em nossos braços , fazendo do seu canto , ou do seu urro , a música do nosso prazer ...
Trá- , eu e tu , em nossos braços , fazendo do seu canto , ou do seu urro , a música do nosso prazer ...

58-375:
Rirse- ia a noite inteira , talvez , se não fosse interrompido pelo Rafael , que o vinha chamar para tomar chá .
Rirse- a noite inteira , talvez , se não fosse interrompido pelo Rafael , que o vinha chamar para tomar chá .

195-5176:
Dirse- ia que o próprio feto repercutiu a sensação e abençoou o pai .
Dirse- que o próprio feto repercutiu a sensação e abençoou o pai .

200-108:
Perorou José Vargas .
José Vargas .




## Preparar material que será anotado pelo UDPipe

In [12]:
count_x85 = 0
for i in range(len(OBRAS_obras)):
        for s in range(len(OBRAS_obras[i]['sentences'])):
            for t in range(len(OBRAS_obras[i]['sentences'][s])):
                if OBRAS_obras[i]['sentences'][s][t]['forma'] == "\x85":
                    OBRAS_obras[i]['sentences'][s][t]['forma'] = OBRAS_obras[i]['sentences'][s][t]['forma'].replace("\x85", "__INDEF__")
                    count_x85 += 1
                    
print(f"{count_x85} caracteres especiais no arquivo foram substituidos por __INDEF__")

51 caracteres especiais no arquivo foram substituidos por __INDEF__


In [13]:
conllu = ""
for i in range(len(OBRAS_obras)):
    for s in range(len(OBRAS_obras[i]['sentences'])):
        conllu += "# sent_id = " + OBRAS_obras[i]['obra'][0].strip().replace(" ", "_") + "-" + str(i) + ":" + str(s)
        conllu += "\n# tituloobra = " + OBRAS_obras[i]['tituloobra'][0]
        conllu += "\n# autor = " + OBRAS_obras[i]['autor'][0]
        conllu += "\n# text = " + " ".join([x['forma'] for x in OBRAS_obras[i]['sentences'][s]])
        conllu += "\n" + "\n".join([x['forma'] + "\t" + "\t".join("_________") for x in OBRAS_obras[i]['sentences'][s]])
        conllu += "\n\n"

print("\n\n".join(conllu.split("\n\n")[:5]))
with open("corpoOBRAS_tokenizado.conllu", "w") as f:
    f.write(conllu)

# sent_id = A_Mortalha_de_Alzira_Prosa:romance_AA_1894_naturalismo_realismo_romantismo_masc-0:0
# tituloobra = A Mortalha de Alzira
# autor = Aluísio Azevedo
# text = A MORTALHA DE ALZIRA
A	_	_	_	_	_	_	_	_	_
MORTALHA	_	_	_	_	_	_	_	_	_
DE	_	_	_	_	_	_	_	_	_
ALZIRA	_	_	_	_	_	_	_	_	_


## Anotar com o UDPipe

In [14]:
! python3 udpipe_vertical.py portuguese-bosque-ud-2.4-190531.udpipe corpoOBRAS_tokenizado.conllu corpoUDPIPE24r.conllu

Loading UDPipe model: done.


In [26]:
! python3 udpipe_vertical.py bosqueud_2.5_workbench.udpipe corpoOBRAS_tokenizado.conllu corpoUDPIPE25wb.conllu

Loading UDPipe model: done.


## Colocando gênero nos flat:name

In [ ]:
import estrutura_ud
ud24r, ud25wb = estrutura_ud.Corpus(recursivo=True), estrutura_ud.Corpus(recursivo=True)
ud24r.load('corpoUDPIPE24r.conllu')
ud25wb.load('corpoUDPIPE25wb.conllu')

def find_gender():
    for sentid, sentence in ud24r.sentences.items():
        for token in sentence.tokens:
            if token.deprel == "flat:name" and "Gender=" in token.head_token.misc:
                print(token.to_str())
                token.misc = token.misc.split("|")
                token.misc = [x for x in token.misc if not x.startswith("Gender=")]
                token.misc.append("Gender=" + token.head_token.misc.split("Gender=")[1].split("|")[0])
                token.misc = "|".join(sorted(token.misc))
                print(token.to_str())
                return ""
find_gender()

## Estrutura de dados do UDPIPE

In [28]:
with open("corpoUDPIPE24r.conllu") as f:
    UDPIPE24r = f.read()
print(UDPIPE24r.split("\n\n")[0])
print(" ")
with open("corpoUDPIPE25wb.conllu") as f:
    UDPIPE25wb = f.read()
print(UDPIPE25wb.split("\n\n")[0])

# text = A MORTALHA DE ALZIRA
# autor = Aluísio Azevedo
# tituloobra = A Mortalha de Alzira
# sent_id = A_Mortalha_de_Alzira_Prosa:romance_AA_1894_naturalismo_realismo_romantismo_masc-0:0
1	A	o	DET	_	Definite=Def|Gender=Fem|Number=Sing|PronType=Art	2	det	_	_
2	MORTALHA	MORTALHA	PROPN	_	Gender=Fem|Number=Sing	0	root	_	_
3	DE	DE	PROPN	_	Number=Sing	2	flat:name	_	_
4	ALZIRA	ALZIRA	PROPN	_	Number=Sing	2	flat:name	_	_
# text = A MORTALHA DE ALZIRA
# autor = Aluísio Azevedo
# tituloobra = A Mortalha de Alzira
# sent_id = A_Mortalha_de_Alzira_Prosa:romance_AA_1894_naturalismo_realismo_romantismo_masc-0:0
1	A	o	DET	_	Definite=Def|Gender=Fem|Number=Sing|PronType=Art	2	det	_	_
2	MORTALHA	MORTALHA	PROPN	_	Gender=Fem|Number=Sing	0	root	_	_
3	DE	DE	PROPN	_	Number=Sing	2	flat:name	_	_
4	ALZIRA	ALZIRA	PROPN	_	Number=Sing	2	flat:name	_	_


In [ ]:
obras = {}
for obra in re.findall(r"# sent_id = .*-(.*?):", UDPIPE24r):
    if not int(obra) in obras:
        obras[int(obra)] = {'sentences': []}

for sentence in UDPIPE24r.split("\n\n"):
    if sentence:
        obras[int(sentence.split("# sent_id = ")[1].split("\n")[0].rsplit("-", 1)[1].split(":")[0])]['obra'] = sentence.split("# sent_id = ")[1].split("\n")[0].rsplit("-", 1)[0]
        obras[int(sentence.split("# sent_id = ")[1].split("\n")[0].rsplit("-", 1)[1].split(":")[0])]['tituloobra'] = sentence.split("# tituloobra = ")[1].split("\n")[0]
        obras[int(sentence.split("# sent_id = ")[1].split("\n")[0].rsplit("-", 1)[1].split(":")[0])]['autor'] = sentence.split("# autor = ")[1].split("\n")[0]
        obras[int(sentence.split("# sent_id = ")[1].split("\n")[0].rsplit("-", 1)[1].split(":")[0])]['sentences'].append([{
                    'forma': z.split("\t")[1],
                    'lema': z.split("\t")[2],
                    'pos': z.split("\t")[3],
                    'genero': z.split("\t")[5].split("Gender=")[1].split("|")[0] if "Gender=" in z.split("\t")[5] else "0",
                } for z in sentence.splitlines() if len(z.split("\t")) > 5])

UDPIPE24r_obras = [{
        'obra': obras[x]['obra'],
        'tituloobra': obras[x]['tituloobra'],
        'autor': obras[x]['autor'],
        'sentences': obras[x]['sentences'],
    } for x in obras]

UDPIPE24r_obras[0]

In [ ]:
obras = {}
for obra in re.findall(r"# sent_id = .*-(.*?):", UDPIPE25wb):
    if not int(obra) in obras:
        obras[int(obra)] = {'sentences': []}

for sentence in UDPIPE25wb.split("\n\n"):
    if sentence:
        obras[int(sentence.split("# sent_id = ")[1].split("\n")[0].rsplit("-", 1)[1].split(":")[0])]['obra'] = sentence.split("# sent_id = ")[1].split("\n")[0].rsplit("-", 1)[0]
        obras[int(sentence.split("# sent_id = ")[1].split("\n")[0].rsplit("-", 1)[1].split(":")[0])]['tituloobra'] = sentence.split("# tituloobra = ")[1].split("\n")[0]
        obras[int(sentence.split("# sent_id = ")[1].split("\n")[0].rsplit("-", 1)[1].split(":")[0])]['autor'] = sentence.split("# autor = ")[1].split("\n")[0]
        obras[int(sentence.split("# sent_id = ")[1].split("\n")[0].rsplit("-", 1)[1].split(":")[0])]['sentences'].append([{
                    'forma': z.split("\t")[1],
                    'lema': z.split("\t")[2],
                    'pos': z.split("\t")[3],
                    'genero': z.split("\t")[5].split("Gender=")[1].split("|")[0] if "Gender=" in z.split("\t")[5] else "0",
                } for z in sentence.splitlines() if len(z.split("\t")) > 5])

UDPIPE25wb_obras = [{
        'obra': obras[x]['obra'],
        'tituloobra': obras[x]['tituloobra'],
        'autor': obras[x]['autor'],
        'sentences': obras[x]['sentences'],
    } for x in obras]

UDPIPE25wb_obras[0]

### Comparar os três corpora

In [ ]:
print(len(PALAVRAS_obras))
print(len(OBRAS_obras))
print(len(UDPIPE24r_obras))
print(len(UDPIPE25wb_obras))

sentencas_diferentes = False
for i in range(len(PALAVRAS_obras)):
    if len(PALAVRAS_obras[i]['sentences']) != len(OBRAS_obras[i]['sentences']) or len(OBRAS_obras[i]['sentences']) != len(UDPIPE24r_obras[i]['sentences']) or len(OBRAS_obras[i]['sentences']) != len(UDPIPE25wb_obras[i]['sentences']):
        sentencas_diferentes = True
        print(f"{i} - {PALAVRAS_obras[i]['tituloobra']}")
        print(f"PALAVRAS ({len(PALAVRAS_obras[i]['sentences'])}) / ACDC ({len(OBRAS_obras[i]['sentences'])}) / UDPIPE24r ({len(UDPIPE24r_obras[i]['sentences'])}) / UDPIPE25wb ({len(UDPIPE25wb_obras[i]['sentences'])})")
        
if not sentencas_diferentes:
    print('Não há obras com número de sentenças diferente')

## Normalizar as formas de dizer o gênero

In [ ]:
lista_genero_PALAVRAS = []
lista_genero_OBRAS = []
lista_genero_UDPIPE24r = []
lista_genero_UDPIPE25wb = []

generos_PALAVRAS = [c['genero'] for a in PALAVRAS_obras for b in a['sentences'] for c in b]
[lista_genero_PALAVRAS.append(x) for x in generos_PALAVRAS if not x in lista_genero_PALAVRAS]
print(lista_genero_PALAVRAS)

generos_OBRAS = [c['genero'] for a in OBRAS_obras for b in a['sentences'] for c in b]
[lista_genero_OBRAS.append(x) for x in generos_OBRAS if not x in lista_genero_OBRAS]
print(lista_genero_OBRAS)

generos_UDPIPE24r = [c['genero'] for a in UDPIPE24r_obras for b in a['sentences'] for c in b]
[lista_genero_UDPIPE24r.append(x) for x in generos_UDPIPE24r if not x in lista_genero_UDPIPE24r]
print(lista_genero_UDPIPE24r)

generos_UDPIPE25wb = [c['genero'] for a in UDPIPE25wb_obras for b in a['sentences'] for c in b]
[lista_genero_UDPIPE25wb.append(x) for x in generos_UDPIPE25wb if not x in lista_genero_UDPIPE25wb]
print(lista_genero_UDPIPE25wb)

In [ ]:
mudanca_de_genero = {
    'F': 'F', 
    '0': '0', 
    'M': 'M', 
    'M/F': 'M/F', 
    'M/F+M': 'M', 
    '+F': 'F', 
    'M+M/F': 'M', 
    'M/F+M/F': 'M/F', 
    'F+M/F': 'F', 
    'M/F+F': 'F', 
    '+M': 'M', 
    'F+F': 'F', 
    'M/F+0': '0', 
    'M+M': 'M', 
    'SUP': 'SUP', 
    'F_P_V_uninfl_INF': 'F', 
    '3P': '3P', 
    '2S': '2S', 
    'M_S_V_uninfl_INF': 'M', 
    'M/F_2S_DAT_PERS_M': 'M/F', 
    'M/F_3S/P_DAT_PERS_M': 'M/F', 
    '+0': '0', 
    'M/S': 'M', 
    'F+M': 'F+M', 
    'm': 'M', 
    'M/F_1S_DAT_PERS_F': 'M/F', 
    'M_1S_DAT_PERS_F': 'M', 
    'M/F_1S_DAT_PERS_M': 'M/F',
    '3S': '3S', 
    'F"': 'F', 
    '1S': '1S', 
    'Fem': 'F',
    'Masc': 'M', 
    'Unsp': 'M/F'
}

for a in range(len(OBRAS_obras)):
    for b in range(len(OBRAS_obras[a]['sentences'])):
        for c in range(len(OBRAS_obras[a]['sentences'][b])):
            OBRAS_obras[a]['sentences'][b][c]['genero'] = mudanca_de_genero[OBRAS_obras[a]['sentences'][b][c]['genero']]
            PALAVRAS_obras[a]['sentences'][b][c]['genero'] = mudanca_de_genero[PALAVRAS_obras[a]['sentences'][b][c]['genero']]
            UDPIPE24r_obras[a]['sentences'][b][c]['genero'] = mudanca_de_genero[UDPIPE24r_obras[a]['sentences'][b][c]['genero']]
            UDPIPE25wb_obras[a]['sentences'][b][c]['genero'] = mudanca_de_genero[UDPIPE25wb_obras[a]['sentences'][b][c]['genero']]
            
print('\nCom a operação de mudança de gênero:')
lista_genero_PALAVRAS = []
lista_genero_OBRAS = []
lista_genero_UDPIPE24r = []
lista_genero_UDPIPE25wb = []

generos_PALAVRAS = [c['genero'] for a in PALAVRAS_obras for b in a['sentences'] for c in b]
[lista_genero_PALAVRAS.append(x) for x in generos_PALAVRAS if not x in lista_genero_PALAVRAS]
print(lista_genero_PALAVRAS)

generos_OBRAS = [c['genero'] for a in OBRAS_obras for b in a['sentences'] for c in b]
[lista_genero_OBRAS.append(x) for x in generos_OBRAS if not x in lista_genero_OBRAS]
print(lista_genero_OBRAS)

generos_UDPIPE24r = [c['genero'] for a in UDPIPE24r_obras for b in a['sentences'] for c in b]
[lista_genero_UDPIPE24r.append(x) for x in generos_UDPIPE24r if not x in lista_genero_UDPIPE24r]
print(lista_genero_UDPIPE24r)

generos_UDPIPE25wb = [c['genero'] for a in UDPIPE25wb_obras for b in a['sentences'] for c in b]
[lista_genero_UDPIPE25wb.append(x) for x in generos_UDPIPE25wb if not x in lista_genero_UDPIPE25wb]
print(lista_genero_UDPIPE25wb)

In [19]:
generos_aceitaveis = ['F', '0', 'M', 'M/F']

## Cálculos de comparação

### Características da comparação

In [20]:
from datetime import datetime

PROP_OBRAS = len([token
    for obra in OBRAS_obras
        for sentence in obra['sentences']
            for token in sentence if token['pos'] == "PROP"])

PROP_PALAVRAS = len([token
    for obra in PALAVRAS_obras
        for sentence in obra['sentences']
            for token in sentence if token['pos'] == "PROP"])

PROP_UDPIPE24r = len([token
    for obra in UDPIPE24r_obras
        for sentence in obra['sentences']
            for token in sentence if token['pos'] == "PROPN"])

PROP_UDPIPE25wb = len([token
    for obra in UDPIPE25wb_obras
        for sentence in obra['sentences']
            for token in sentence if token['pos'] == "PROPN"])

### Acertos de nome próprio

In [21]:
dicOBRAS_PROP = []
dicPALAVRAS_PROP = []
dicUDPIPE_PROP = []
PALAVRAS_acertos_PROP = 0
UDPIPE24r_acertos_PROP = 0
UDPIPE25wb_acertos_PROP = 0
acerto_ambos = []
for o, obra in enumerate(OBRAS_obras):
    for s, sentence in enumerate(obra['sentences']):
        for t, token in enumerate(sentence):
            tem_PALAVRAS = False
            tem_UDPIPE24r = False
            tem_UDPIPE25wb = False
            if OBRAS_obras[o]['sentences'][s][t]['pos'] == 'PROP':
                dicOBRAS_PROP.append({'o': o, 's': s, 't': t})
                if PALAVRAS_obras[o]['sentences'][s][t]['pos'] == 'PROP':
                    PALAVRAS_acertos_PROP += 1
                    tem_PALAVRAS = True
                if UDPIPE24r_obras[o]['sentences'][s][t]['pos'] == 'PROPN':
                    UDPIPE24r_acertos_PROP += 1
                    tem_UDPIPE24r = True
                if UDPIPE25wb_obras[o]['sentences'][s][t]['pos'] == 'PROPN':
                    UDPIPE25wb_acertos_PROP += 1
                    tem_UDPIPE25wb = True
                if tem_PALAVRAS and tem_UDPIPE:
                    acerto_ambos.append({'o': o, 's': s, 't': t})
OBRAS_total_PROP = len(dicOBRAS_PROP)

### Acertos de gênero

In [22]:
acerto_ambos = [{'o': x['o'], 's': x['s'], 't': x['t'], 'g': OBRAS_obras[x['o']]['sentences'][x['s']][x['t']]['genero']} for x in acerto_ambos if OBRAS_obras[x['o']]['sentences'][x['s']][x['t']]['genero'] in generos_aceitaveis]
nomes_comparaveis = len(acerto_ambos)

acertoGenero_PALAVRAS = 0
acertoGenero_UDPIPE24r = 0
acertoGenero_UDPIPE25wb = 0
for token in acerto_ambos:
    if PALAVRAS_obras[token['o']]['sentences'][token['s']][token['t']]['genero'] == token['g']:
        acertoGenero_PALAVRAS += 1
    if UDPIPE24r_obras[token['o']]['sentences'][token['s']][token['t']]['genero'] == token['g']:
        acertoGenero_UDPIPE24r += 1
    if UDPIPE25wb_obras[token['o']]['sentences'][token['s']][token['t']]['genero'] == token['g']:
        acertoGenero_UDPIPE25wb += 1

## HTML

In [23]:
html = "<style>body { width:40%; margin:auto; padding-top:10px; } td, th { padding:5px; } table { margin:auto; }</style>"
html += "<title>Comparação entre a anotação de nomes próprios no OBras realizada pelo PALAVRAS e pelo UDPipe</title>"
html += "<h1>Comparação entre a anotação de nomes próprios no OBras realizada pelo PALAVRAS e pelo UDPipe</h1><hr>"
html += "<table border='1'>"
html += "<tr><th>Características</th><th>PALAVRAS</th><th>UDPIPE24r</th><th>UDPIPE25wb</th><th>GOLDEN</th></td></tr>"
html += "<tr><th>Obras</th><td>{}</td><td>{}</td><td>{}</td><td>{}</td></tr>".format(
    len(PALAVRAS_obras), 
    len(UDPIPE24r_obras), 
    len(UDPIPE25wb_obras), 
    len(OBRAS_obras)
)
html += "<tr><th>Sentenças</th><td>{}</td><td>{}</td><td>{}</td><td>{}</td></tr>".format(
    sum([len(x['sentences']) for x in PALAVRAS_obras]), 
    sum([len(x['sentences']) for x in UDPIPE24r_obras]), 
    sum([len(x['sentences']) for x in UDPIPE25wb_obras]), 
    sum([len(x['sentences']) for x in OBRAS_obras])
)
html += "<tr><th>Tokens</th><td>{}</td><td>{}</td><td>{}</td><td>{}</td></tr>".format(
    sum([len(x) for y in PALAVRAS_obras for x in y['sentences']]), 
    sum([len(x) for y in UDPIPE24r_obras for x in y['sentences']]), 
    sum([len(x) for y in UDPIPE25wb_obras for x in y['sentences']]), 
    sum([len(x) for y in OBRAS_obras for x in y['sentences']])
)
html += "<tr><th>Nomes próprios</th><td>{}</td><td>{}</td><td>{}</td><td>{}</td></tr>".format(
    PROP_PALAVRAS, 
    PROP_UDPIPE24r, 
    PROP_UDPIPE25wb, 
    PROP_OBRAS
)
html += "</table>"

html += "<br><table border='1'>"
html += "<tr><th></th><th>PALAVRAS</th><th>UDPIPE24r</th><th>UDPIPE25wb</th><th>GOLDEN</th></tr>"
html += "<tr><th>Acertos de nome próprio</th><td>{}</td><td>{}</td><td>{}</td><td>{}</td></tr>".format(
    f"{round((PALAVRAS_acertos_PROP/OBRAS_total_PROP)*100)}% ({PALAVRAS_acertos_PROP})", 
    f"{round((UDPIPE24r_acertos_PROP/OBRAS_total_PROP)*100)}% ({UDPIPE24r_acertos_PROP})", 
    f"{round((UDPIPE25wb_acertos_PROP/OBRAS_total_PROP)*100)}% ({UDPIPE25wb_acertos_PROP})", 
    OBRAS_total_PROP
)
html += "</table>"

html += "<br><table border='1'>"
html += "<tr><th></th><th>PALAVRAS</th><th>UDPIPE24r</th><th>UDPIPE25wb</th><th>Nomes próprios comparáveis</th></tr>"
html += "<tr><th>Acertos de gênero</th><td>{}</td><td>{}</td><td>{}</td><td>{}</td></tr>".format(
    f"{round((acertoGenero_PALAVRAS/nomes_comparaveis)*100)}% ({acertoGenero_PALAVRAS})", 
    f"{round((acertoGenero_UDPIPE24r/nomes_comparaveis)*100)}% ({acertoGenero_UDPIPE24r})", 
    f"{round((acertoGenero_UDPIPE25wb/nomes_comparaveis)*100)}% ({acertoGenero_UDPIPE25wb})", 
    nomes_comparaveis
)
html += "</table><br>"

for i, token in enumerate(acerto_ambos):
    html += "<hr><table style='text-align:center'>"
    html += "<tr><td colspan='4'>{} / {}<br>{}</td></tr>".format(str(i+1), len(acerto_ambos), " ".join([x['forma'] for x in UDPIPE24r_obras[token['o']]['sentences'][token['s']]]).replace(UDPIPE24r_obras[token['o']]['sentences'][token['s']][token['t']]['forma'], '<b>' + UDPIPE24r_obras[token['o']]['sentences'][token['s']][token['t']]['forma'] + "</b>"))
    html += "<tr><th>PALAVRAS</th><th>UDPIPE24r</th><th>UDPIPE25wb</th><th>GOLDEN</th></tr>"
    html += "<tr><td><font color='{cor_palavras}'>{}</font></td><td><font color='{cor_udpipe24r}'>{}</font></td><td><font color='{cor_udpipe25wb}'>{}</font></td><td>{}</td></tr>".format(
        PALAVRAS_obras[token['o']]['sentences'][token['s']][token['t']]['genero'], 
        UDPIPE24r_obras[token['o']]['sentences'][token['s']][token['t']]['genero'], 
        UDPIPE25wb_obras[token['o']]['sentences'][token['s']][token['t']]['genero'], 
        OBRAS_obras[token['o']]['sentences'][token['s']][token['t']]['genero'], 
        cor_palavras="red" if PALAVRAS_obras[token['o']]['sentences'][token['s']][token['t']]['genero'] != OBRAS_obras[token['o']]['sentences'][token['s']][token['t']]['genero'] else "green", 
        cor_udpipe24r="red" if UDPIPE_obras[token['o']]['sentences'][token['s']][token['t']]['genero'] != OBRAS_obras[token['o']]['sentences'][token['s']][token['t']]['genero'] else "green", 
        cor_udpipe25wb="red" if UDPIPE25wb_obras[token['o']]['sentences'][token['s']][token['t']]['genero'] != OBRAS_obras[token['o']]['sentences'][token['s']][token['t']]['genero'] else "green"
    )
    html += "</table>"
    if i == 199:
        break

html += "<br><hr>Relatório gerado em " + str(datetime.now()) + "<br><br>"

with open("gender-palavras-udpipe.html", "w") as f:
    f.write(html)